[참고사이트](https://github.com/works-code/word2vec/blob/main/20201018_shopping_cart_recommend.ipynb)

## 라이브러리 정의

In [1]:
import pandas as pd

# 단어 임베딩
from gensim.models import Word2Vec

# View
from matplotlib import rc
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

## 함수 & 데이터 정의

In [2]:
# 그래프 그려주는 함수
def plot_2d_graph(vocabs, xs, ys):
    rc('font', family='AppleGothic')
    plt.rcParams['axes.unicode_minus'] = False
    plt.figure(figsize=(8,6))
    plt.scatter(xs, ys, marker = 'o')
    for i, v in enumerate(vocabs):
        plt.annotate(v, xy=(xs[i], ys[i]))

In [3]:
raw = pd.read_csv('/Users/mac/AIFFEL/hackathon/new_musinsa(upcycling).csv', engine='python')
print(raw.shape)
raw.head()

(715, 12)


,Unnamed: 0,user,color,contents,category,ratings,gender,price,style,up_check,brand,url
0,0,멤버_464f9cc84f7d,기타,면,상의,5,남자,39000,0,0,119레오,https://www.musinsa.com/app/goods/1876547/0
1,1,멤버_464f9cc84f7d,카키,면,하의,5,남자,42000,0,0,119레오,https://www.musinsa.com/app/goods/2396824/0
2,2,멤버_464f9cc84f7d,기타,면,모자,5,남자,59000,0,0,119레오,https://www.musinsa.com/app/goods/2428013/0
3,3,멤버_464f9cc84f7d,블랙,면,상의,5,남자,35000,0,0,119레오,https://www.musinsa.com/app/goods/470357/0
4,4,멤버_464f9cc84f7d,아이보리,면 아라미드,가방,5,남자,48000,1,1,119레오,https://www.musinsa.com/app/goods/1766332/0


In [4]:
# 불필요한 컬럼 제거
raw.drop(['Unnamed: 0'], axis=1, inplace=True)
raw.head()

,user,color,contents,category,ratings,gender,price,style,up_check,brand,url
0,멤버_464f9cc84f7d,기타,면,상의,5,남자,39000,0,0,119레오,https://www.musinsa.com/app/goods/1876547/0
1,멤버_464f9cc84f7d,카키,면,하의,5,남자,42000,0,0,119레오,https://www.musinsa.com/app/goods/2396824/0
2,멤버_464f9cc84f7d,기타,면,모자,5,남자,59000,0,0,119레오,https://www.musinsa.com/app/goods/2428013/0
3,멤버_464f9cc84f7d,블랙,면,상의,5,남자,35000,0,0,119레오,https://www.musinsa.com/app/goods/470357/0
4,멤버_464f9cc84f7d,아이보리,면 아라미드,가방,5,남자,48000,1,1,119레오,https://www.musinsa.com/app/goods/1766332/0


In [5]:
# feature 합치기
cols = ['color', 'contents', 'category', 'ratings', 'gender', 'price', 'style']
raw['features'] = raw[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
raw.drop(cols, axis=1, inplace=True)
raw

,user,up_check,brand,url,features
0,멤버_464f9cc84f7d,0,119레오,https://www.musinsa.com/app/goods/1876547/0,기타 면 상의 5 남자 39000 0
1,멤버_464f9cc84f7d,0,119레오,https://www.musinsa.com/app/goods/2396824/0,카키 면 하의 5 남자 42000 0
2,멤버_464f9cc84f7d,0,119레오,https://www.musinsa.com/app/goods/2428013/0,기타 면 모자 5 남자 59000 0
3,멤버_464f9cc84f7d,0,119레오,https://www.musinsa.com/app/goods/470357/0,블랙 면 상의 5 남자 35000 0
4,멤버_464f9cc84f7d,1,119레오,https://www.musinsa.com/app/goods/1766332/0,아이보리 면 아라미드 가방 5 남자 48000 1
...,...,...,...,...,...
710,허싱싱,1,플리츠마마,https://www.musinsa.com/app/goods/2041227/0,기타 폴리에스터 폴리우레탄 가방 5 여자 59000 0
711,허싱싱,0,플리츠마마,https://www.musinsa.com/app/goods/2196725/0,아이보리 울 아크릴 모자 5 여자 35000 0
712,석재은,0,플리츠마마,https://www.musinsa.com/app/goods/2122170/0,베이지 아크릴 모자 5 여자 29000 0
713,석재은,0,플리츠마마,https://www.musinsa.com/app/goods/2064480/0,블랙 면 폴리에스터 상의 5 여자 138000 0


In [6]:
# 컬럼 순서 정리
raw = raw[['user', 'features', 'up_check', 'brand', 'url']]
raw.head()

,user,features,up_check,brand,url
0,멤버_464f9cc84f7d,기타 면 상의 5 남자 39000 0,0,119레오,https://www.musinsa.com/app/goods/1876547/0
1,멤버_464f9cc84f7d,카키 면 하의 5 남자 42000 0,0,119레오,https://www.musinsa.com/app/goods/2396824/0
2,멤버_464f9cc84f7d,기타 면 모자 5 남자 59000 0,0,119레오,https://www.musinsa.com/app/goods/2428013/0
3,멤버_464f9cc84f7d,블랙 면 상의 5 남자 35000 0,0,119레오,https://www.musinsa.com/app/goods/470357/0
4,멤버_464f9cc84f7d,아이보리 면 아라미드 가방 5 남자 48000 1,1,119레오,https://www.musinsa.com/app/goods/1766332/0


In [7]:
# 데이터 프레임 재정의
new_df = raw[['user', 'features', 'brand']]
new_df.head()

,user,features,brand
0,멤버_464f9cc84f7d,기타 면 상의 5 남자 39000 0,119레오
1,멤버_464f9cc84f7d,카키 면 하의 5 남자 42000 0,119레오
2,멤버_464f9cc84f7d,기타 면 모자 5 남자 59000 0,119레오
3,멤버_464f9cc84f7d,블랙 면 상의 5 남자 35000 0,119레오
4,멤버_464f9cc84f7d,아이보리 면 아라미드 가방 5 남자 48000 1,119레오


In [8]:
# user별로 feature를 합친다.
new_df = new_df.groupby(['user'])['features'].agg({'unique'})
new_df

,unique
user,
*****\\,"[블랙 면 종이 모자 5 여자 39000 0, 그레이 폴리에스터 레이온 폴리우레탄 ..."
Air젤,"[화이트 None 상의 5 남자 126000 0, 블루 면 하의 5 남자 10300..."
Bamyerinin,"[네이비 울 나일론 모자 5 여자 39000 0, 블루 폴리에스터 폴리우레탄 가방 ..."
J flora,"[블랙 면 린넨 하의 5 남자 53000 0, 블랙 면 폴리에스터 상의 5 남자 3..."
K.Hobum,"[네이비 면 모자 5 남자 29000 0, 블랙 나일론 가방 5 남자 68000 0..."
...,...
항정살보단가브리살,"[블랙 리사이클폴리에스터 가방 5 남자 35000 0, 베이지 면 모자 5 남자 4..."
허싱싱,"[기타 폴리에스터 폴리우레탄 가방 5 여자 59000 0, 아이보리 울 아크릴 모자..."
헐크랑,"[블랙 면 상의 5 여자 59000 0, 블랙 폴리에스터 폴리우레탄 가방 5 여자 ..."


In [9]:
# ',' 를 공백으로 대체

for k, i in enumerate(new_df['unique']):
    new_df['unique'][k] = " ".join(str(_) for _ in i) 
    
new_df



,unique
user,
*****\\,블랙 면 종이 모자 5 여자 39000 0 그레이 폴리에스터 레이온 폴리우레탄 상의...
Air젤,화이트 None 상의 5 남자 126000 0 블루 면 하의 5 남자 103000 ...
Bamyerinin,네이비 울 나일론 모자 5 여자 39000 0 블루 폴리에스터 폴리우레탄 가방 5 ...
J flora,블랙 면 린넨 하의 5 남자 53000 0 블랙 면 폴리에스터 상의 5 남자 318...
K.Hobum,네이비 면 모자 5 남자 29000 0 블랙 나일론 가방 5 남자 68000 0 블...
...,...
항정살보단가브리살,블랙 리사이클폴리에스터 가방 5 남자 35000 0 베이지 면 모자 5 남자 430...
허싱싱,기타 폴리에스터 폴리우레탄 가방 5 여자 59000 0 아이보리 울 아크릴 모자 5...
헐크랑,블랙 면 상의 5 여자 59000 0 블랙 폴리에스터 폴리우레탄 가방 5 여자 59...


## 단어 토큰화

In [10]:
import nltk
from nltk.tokenize import TreebankWordTokenizer

# TreebankWord 토크나이저 사용
tb_tokenizer = TreebankWordTokenizer()

def tokenizer(row):
    return tb_tokenizer.tokenize(row)

In [11]:
new_df['unique'] = new_df['unique'].apply(tokenizer)
new_df.head()

,unique
user,
*****\\,"[블랙, 면, 종이, 모자, 5, 여자, 39000, 0, 그레이, 폴리에스터, 레..."
Air젤,"[화이트, None, 상의, 5, 남자, 126000, 0, 블루, 면, 하의, 5..."
Bamyerinin,"[네이비, 울, 나일론, 모자, 5, 여자, 39000, 0, 블루, 폴리에스터, ..."
J flora,"[블랙, 면, 린넨, 하의, 5, 남자, 53000, 0, 블랙, 면, 폴리에스터,..."
K.Hobum,"[네이비, 면, 모자, 5, 남자, 29000, 0, 블랙, 나일론, 가방, 5, ..."


In [12]:
values_list = new_df['unique'].values.tolist()
len(values_list)

97

## 단어 임베딩

In [13]:
import numpy as np

vector_df = pd.DataFrame(columns=['idx', 'word_vec'])

for i, k in enumerate(new_df['unique']):

    model = Word2Vec([k], 
                     vector_size=20, window=5, 
                     min_count=1, epochs=200, 
                     sg=0, workers=4)
    # 단어의 벡터를 구한다.
    word_vectors = model.wv
    vocabs = word_vectors.index_to_key
    word_vectors_list = [word_vectors[v] for v in vocabs]
    vector_df = vector_df.append(pd.DataFrame([[i, word_vectors_list]], 
                                              columns=['idx','word_vec']), 
                                 ignore_index=True)

In [14]:
vector_df

,idx,word_vec
0,0,"[[-0.21225461, -0.20802173, 0.48381108, -0.135..."
1,1,"[[-0.07025324, 0.03248649, 0.08816097, 0.04732..."
2,2,"[[-0.14733347, 0.12730415, 0.16985007, 0.06836..."
3,3,"[[-0.4766998, 0.1374626, 0.32661548, -0.238854..."
4,4,"[[-0.045516845, 0.023472229, 0.031406965, 0.04..."
...,...,...
92,92,"[[-0.41012666, 0.279256, 0.12115218, -0.074502..."
93,93,"[[-0.01174727, 0.004926758, 0.02598539, 0.0415..."
94,94,"[[-0.054519024, 0.031231469, 0.030536884, 0.03..."
95,95,"[[-0.479609, 0.051198006, 0.5382129, 0.2164029..."


In [15]:
vector_list = vector_df['word_vec']

In [17]:
vector_list

35

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
# 코사인 유사도 매트릭스

cosine_matrix = cosine_similarity(vector_list,vector_list)

print(cosine_matrix.shape)

In [ ]:
new_df[0:11]

In [ ]:
# size : 벡터차원
# window : 주변 단어 수
# iter : 반복 횟수
# sg : 모델 (0: CBow | 1: skip-gram)
# workers : 동시에 처리할 작업 수

model = Word2Vec(new_df['features'], vector_size=20, window=5, min_count=1, epochs=200, sg=0, workers=4)
# 단어의 벡터를 구한다.
word_vectors = model.wv
word_vectors
vocabs = word_vectors.index_to_key


In [ ]:

vocabs = word_vectors.index_to_key
word_vectors_list = [word_vectors[v] for v in vocabs]
word_vectors_list[:10]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
# 코사인 유사도 매트릭스
cosine_matrix = cosine_similarity(word_vectors_list,word_vectors_list)

print(cosine_matrix.shape)

In [ ]:
import numpy as np
np.round(cosine_matrix, 4)